In [3]:
# compenency query is for fasc diam, nerv diam, and lateriality
# obj = 55c5b69c-a5b8-4881-a105-e4048af26fa5
# objid = 20720c2e-83fb-4454-bef1-1ce6a97fa748

In [11]:
import ast

import pandas as pdfdsadaasdfd
import uuid

from sqlalchemy import inspect, UniqueConstraint
from sqlalchemy.orm import Session
from sqlalchemy.inspection import inspect
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm.exc import NoResultFound

from quantdb.ingest import Queries, InternalIds
from quantdb.client import get_session
from sqlalchemy.sql.elements import ClauseElement
from quantdb.ingest_tabular.models import (
    # Ingest,
    # Unit,
    Objects,
    Units,
    Addresses,
    Aspects,
    DescriptorsInst,
    DescriptorsQuant,
    ObjDescInst,
    ObjDescQuant,
    ValuesInst,
    ValuesQuant,
    ValuesCat,
    ControlledTerms,
    t_dataset_object,
)
from sqlalchemy.orm.exc import NoResultFound
import pandas as pd

df = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
df = df.T.reset_index(drop=True)
df["id_sub"] = "sub-" + df.sub_sam
df.index = df.index + 1
# df.head()


def object_as_dict(obj):
    return {
        c.key: getattr(obj, c.key)
        for c in inspect(obj).mapper.column_attrs
        if getattr(obj, c.key)
    }


def get_or_create(session, obj, back_populate=None):
    model = obj.__class__
    data = object_as_dict(obj)
    instance = session.query(model).filter_by(**data).one_or_none()
    if instance:
        return instance
    else:
        params = {
            k: v for k, v in data.items() if not isinstance(v, ClauseElement)
        }
        instance = model(**params)
        try:
            session.add(instance)
            session.commit()
            if back_populate:
                for attr, value in back_populate.items():
                    setattr(instance, attr, value)
                session.commit()
        except Exception:
            session.rollback()
            instance = session.query(model).filter_by(**data).one()
            return instance
        else:
            return instance


def get_constraint_columns(model):
    """
    Returns the columns of the unique constraints for a SQLAlchemy model.

    Args:
        model: The SQLAlchemy model.

    Returns:
        A list of lists of column objects.
    """

    constraints = [
        constraint
        for constraint in model.__mapper__.tables[0].constraints
        if isinstance(constraint, UniqueConstraint)
    ]

    return [
        [c.name for c in list(constraint.columns)] for constraint in constraints
    ]


def query_by_constraints(db, obj):
    # print("made it")
    filter_criteria = object_as_dict(obj)

    existing_obj = None
    for cols in get_constraint_columns(obj):

        # print(type(obj), "cols", cols)
        # print("cols", filter_criteria)
        if set(cols) != (set(cols) & set(filter_criteria.keys())):
            continue

        filter_criteria = {
            key: value for key, value in filter_criteria.items() if key in cols
        }

        # print("filter using", filter_criteria)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        if existing_obj:
            # print("found!!")
            return existing_obj
        # print("not found")


def back_populate_tables(db: Session, obj) -> object:
    """
    Recursively back-populates objects in the database,
    retrieving existing objects instead of failing on duplicates.
    Iterates through all relationships of the object for both
    initial population and recursive calls.
    Includes a complete rollback if any commit fails.
    Filters by all primary key columns of the object.
    Handles cases where the child object is not a valid ORM instance.

    Args:
        db: The database session.
        obj: The ORM model object.

    Returns:
        The ORM model object, with its related objects back-populated.
    """

    mapper = obj.__mapper__  # Get the mapper for the object
    # print(obj)
    try:
        # Iterate through all relationships of the object
        for relationship_prop in mapper.relationships:
            if (
                relationship_prop.direction.name == "MANYTOONE"
                or relationship_prop.direction.name == "MANYTOMANY"
            ):
                parent = getattr(obj, relationship_prop.key)
                if parent:
                    filter_criteria = object_as_dict(parent)
                    existing_obj = (
                        db.query(type(parent))
                        .filter_by(**filter_criteria)
                        .first()
                    )
                    if existing_obj:
                        for key, value in object_as_dict(existing_obj).items():
                            if hasattr(parent, key):
                                setattr(parent, key, value)
                    # print("populated", object_as_dict(parent))
                    setattr(obj, relationship_prop.key, parent)
                    # db.add(parent)
                    # db.commit()
                    back_populate_tables(db, parent)

        # print("object", type(obj))
        filter_criteria = object_as_dict(obj)
        print(type(obj), filter_criteria)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        print("filter criteria", filter_criteria)
        print("found:", existing_obj)

        if not existing_obj:
            existing_obj = query_by_constraints(db, obj)

        # print("filter using", filter_criteria)
        # existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        # print("is existing??", existing_obj)

        if not existing_obj:
            db.add(obj)
            # db.add(obj)
            # db.flush()
            db.commit()
            # db.refresh(obj)
            print("added new")
        else:
            db.merge(obj)
            db.commit()
            for key, value in object_as_dict(existing_obj).items():
                if hasattr(obj, key):
                    setattr(obj, key, value)
            print("updated existing")
            # print("after update", object_as_dict(existing_obj))

    except IntegrityError as e:
        db.rollback()  # Rollback the transaction in case of an error
        # print(f"Error during commit: {e}")
        raise  # Re-raise the exception to indicate failure

    return obj  # Return the object with its related objects back-populated


session = get_session(echo=False)

object_dataset = Objects(
    id="55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
object_package = Objects(
    id="20720c2e-83fb-4454-bef1-1ce6a97fa748",
    id_type="package",
    id_file=1094489,
    id_internal=None,
    objects_=object_dataset,
)
back_populate_tables(session, object_package)

<class 'quantdb.ingest_tabular.models.Objects'> {'id': UUID('55c5b69c-a5b8-4881-a105-e4048af26fa5'), 'id_type': 'dataset'}
filter criteria {'id': UUID('55c5b69c-a5b8-4881-a105-e4048af26fa5'), 'id_type': 'dataset'}
found: <Objects(id=55c5b69c-a5b8-4881-a105-e4048af26fa5, id_type=dataset)>
updated existing
<class 'quantdb.ingest_tabular.models.Objects'> {'id': '20720c2e-83fb-4454-bef1-1ce6a97fa748', 'id_type': 'package', 'id_file': 1094489}
filter criteria {'id': '20720c2e-83fb-4454-bef1-1ce6a97fa748', 'id_type': 'package', 'id_file': 1094489}
found: <Objects(id=20720c2e-83fb-4454-bef1-1ce6a97fa748, id_type=package)>
updated existing


<Objects(id=20720c2e-83fb-4454-bef1-1ce6a97fa748, id_type=package)>

In [10]:
import ast

import pandas as pdfdsadaasdfd
import uuid

from sqlalchemy import inspect, UniqueConstraint
from sqlalchemy.orm import Session
from sqlalchemy.inspection import inspect
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm.exc import NoResultFound

from quantdb.ingest import Queries, InternalIds
from quantdb.client import get_session
from sqlalchemy.sql.elements import ClauseElement
from quantdb.ingest_tabular.models import (
    # Ingest,
    # Unit,
    Objects,
    Units,
    Addresses,
    Aspects,
    DescriptorsInst,
    DescriptorsQuant,
    ObjDescInst,
    ObjDescQuant,
    ValuesInst,
    ValuesQuant,
    ValuesCat,
    ControlledTerms,
    t_dataset_object,
)
from sqlalchemy.orm.exc import NoResultFound
import pandas as pd

df = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
df = df.T.reset_index(drop=True)
df["id_sub"] = "sub-" + df.sub_sam
df.index = df.index + 1
# df.head()


def object_as_dict(obj):
    return {
        c.key: getattr(obj, c.key)
        for c in inspect(obj).mapper.column_attrs
        if getattr(obj, c.key)
    }


def get_or_create(session, obj, back_populate=None):
    model = obj.__class__
    data = object_as_dict(obj)
    instance = session.query(model).filter_by(**data).one_or_none()
    if instance:
        return instance
    else:
        params = {
            k: v for k, v in data.items() if not isinstance(v, ClauseElement)
        }
        instance = model(**params)
        try:
            session.add(instance)
            session.commit()
            if back_populate:
                for attr, value in back_populate.items():
                    setattr(instance, attr, value)
                session.commit()
        except Exception:
            session.rollback()
            instance = session.query(model).filter_by(**data).one()
            return instance
        else:
            return instance


def get_constraint_columns(model):
    """
    Returns the columns of the unique constraints for a SQLAlchemy model.

    Args:
        model: The SQLAlchemy model.

    Returns:
        A list of lists of column objects.
    """

    constraints = [
        constraint
        for constraint in model.__mapper__.tables[0].constraints
        if isinstance(constraint, UniqueConstraint)
    ]

    return [
        [c.name for c in list(constraint.columns)] for constraint in constraints
    ]


def query_by_constraints(db, obj):
    # print("made it")
    filter_criteria = object_as_dict(obj)

    existing_obj = None
    for cols in get_constraint_columns(obj):

        # print(type(obj), "cols", cols)
        # print("cols", filter_criteria)
        if set(cols) != (set(cols) & set(filter_criteria.keys())):
            continue

        filter_criteria = {
            key: value for key, value in filter_criteria.items() if key in cols
        }

        # print("filter using", filter_criteria)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        if existing_obj:
            # print("found!!")
            return existing_obj
        # print("not found")


def back_populate_tables(db: Session, obj) -> object:
    """
    Recursively back-populates objects in the database,
    retrieving existing objects instead of failing on duplicates.
    Iterates through all relationships of the object for both
    initial population and recursive calls.
    Includes a complete rollback if any commit fails.
    Filters by all primary key columns of the object.
    Handles cases where the child object is not a valid ORM instance.

    Args:
        db: The database session.
        obj: The ORM model object.

    Returns:
        The ORM model object, with its related objects back-populated.
    """

    mapper = obj.__mapper__  # Get the mapper for the object
    # print(obj)
    try:
        # Iterate through all relationships of the object
        for relationship_prop in mapper.relationships:
            if (
                relationship_prop.direction.name == "MANYTOONE"
                or relationship_prop.direction.name == "MANYTOMANY"
            ):
                parent = getattr(obj, relationship_prop.key)
                if parent:
                    filter_criteria = object_as_dict(parent)
                    existing_obj = (
                        db.query(type(parent))
                        .filter_by(**filter_criteria)
                        .first()
                    )
                    if existing_obj:
                        for key, value in object_as_dict(existing_obj).items():
                            if hasattr(parent, key):
                                setattr(parent, key, value)
                    # print("populated", object_as_dict(parent))
                    setattr(obj, relationship_prop.key, parent)
                    # db.add(parent)
                    # db.commit()
                    back_populate_tables(db, parent)

        # print("object", type(obj))
        filter_criteria = object_as_dict(obj)
        print(type(obj), filter_criteria)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        print("filter criteria", filter_criteria)
        print("found:", existing_obj)

        if not existing_obj:
            existing_obj = query_by_constraints(db, obj)

        # print("filter using", filter_criteria)
        # existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        # print("is existing??", existing_obj)

        if not existing_obj:
            db.add(obj)
            # db.add(obj)
            # db.flush()
            db.commit()
            # db.refresh(obj)
            print("added new")
        else:
            db.merge(obj)
            db.commit()
            for key, value in object_as_dict(existing_obj).items():
                if hasattr(obj, key):
                    setattr(obj, key, value)
            print("updated existing")
            # print("after update", object_as_dict(existing_obj))

    except IntegrityError as e:
        db.rollback()  # Rollback the transaction in case of an error
        # print(f"Error during commit: {e}")
        raise  # Re-raise the exception to indicate failure

    return obj  # Return the object with its related objects back-populated


session = get_session(echo=False)

object_dataset = Objects(
    id="55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
object_package = Objects(
    id="20720c2e-83fb-4454-bef1-1ce6a97fa748",
    id_type="package",
    id_file=1094489,
    id_internal=None,
    objects_=object_dataset,
)

# number of fascicles

# Dataset
object_dataset = Objects(
    id="55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
object_package = Objects(
    id="20720c2e-83fb-4454-bef1-1ce6a97fa748",
    id_type="package",
    id_file=1094489,
    id_internal=None,
    objects_=object_dataset,
)

# Domain
unit = Units(
    label="unitless",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/micrometer",
)

aspect = Aspects(
    label="count",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/count",
)
# focus of measurement
desc_inst = DescriptorsInst(
    id=16,
    label="nerve-cross-section",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/cross-section",
    # description=None,
)

# how to calculate unit
desc_quant = DescriptorsQuant(
    id=35,
    shape="scalar",
    label="count of fascicles",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)

# back_populate_tables(session, desc_inst)
# back_populate_tables(session, desc_quant)

# Objects
address = Addresses(
    id=1,
    addr_type="constant",
    value_type="single",
)
obj_desc_inst = ObjDescInst(
    objects=object_package,
    descriptors_inst=desc_inst,
    addresses_field=address,
    addr_field=address.id,  # Ensure addr_field is not null
)
obj_desc_quant = ObjDescQuant(
    objects=object_package,
    descriptors_quant=desc_quant,
    addresses_field=address,
    addr_field=address.id,
)

# values type
controlled_term = ControlledTerms(
    label="controlled thing",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
)

back_populate_tables(session, obj_desc_inst)
back_populate_tables(session, obj_desc_quant)

for i, row in df.iterrows():
    id_sub = "sub-" + row.sub_sam[1:].split("-")[0]
    id_sam = "sam-" + row.sub_sam[1:]
    id_formal = id_sam
    # try:
    #     # Check if the entry already exists
    #     existing_values_inst = (
    #         session.query(ValuesInst)
    #         .filter_by(
    #             id_sub=id_sub,
    #             # id_formal=id_formal,
    #             id_sam=id_sam,
    #             objects=object_package,
    #         )
    #         .one()
    #     )
    #     values_inst = existing_values_inst
    # except NoResultFound:
    # If not found, create a new entry
    values_inst = ValuesInst(
        dataset=object_dataset,
        descriptors_inst=desc_inst,
        type="sample",
        id_sub=id_sub,
        id_sam=id_sam,
        id_formal=id_formal,
        local_identifier=id_sam,
        objects=object_package,
    )
    try:
        back_populate_tables(session, values_inst)
    except:
        session.rollback()
        print("rollback")
    # session.add(values_inst)
    # session.commit()

    values_quant = ValuesQuant(
        value=int(row["NFasc"]),
        objects=object_package,
        orig_value=None,
        orig_units=None,
        descriptors_inst=desc_inst,
        descriptors_quant=desc_quant,
        values_inst=values_inst,
        value_blob=row["NFasc"],
    )
    values_quant = back_populate_tables(session, values_quant)
    break

<class 'quantdb.ingest_tabular.models.Addresses'> {'id': 1, 'addr_type': 'constant', 'value_type': 'single'}
filter criteria {'id': 1, 'addr_type': 'constant', 'value_type': 'single'}
found: <quantdb.ingest_tabular.models.Addresses object at 0x176cf1040>
updated existing
<class 'quantdb.ingest_tabular.models.DescriptorsInst'> {'id': 16, 'label': 'nerve-cross-section', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/cross-section'}
filter criteria {'id': 16, 'label': 'nerve-cross-section', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/cross-section'}
found: <quantdb.ingest_tabular.models.DescriptorsInst object at 0x176d3ce60>
updated existing
<class 'quantdb.ingest_tabular.models.Objects'> {'id': UUID('55c5b69c-a5b8-4881-a105-e4048af26fa5'), 'id_type': 'dataset'}
filter criteria {'id': UUID('55c5b69c-a5b8-4881-a105-e4048af26fa5'), 'id_type': 'dataset'}
found: <Objects(id=55c5b69c-a5b8-4881-a105-e4048af26fa5, id_type=dataset)>
upd

In [4]:
import ast

import pandas as pdfdsadaasdfd
import uuid

from sqlalchemy import inspect, UniqueConstraint
from sqlalchemy.orm import Session
from sqlalchemy.inspection import inspect
from sqlalchemy.exc import IntegrityError

from quantdb.ingest import Queries, InternalIds
from quantdb.client import get_session
from sqlalchemy.sql.elements import ClauseElement
from quantdb.ingest_tabular.models import (
    # Ingest,
    # Unit,
    Objects,
    Units,
    Addresses,
    Aspects,
    DescriptorsInst,
    DescriptorsQuant,
    ObjDescInst,
    ObjDescQuant,
    ValuesInst,
    ValuesQuant,
    ValuesCat,
    ControlledTerms,
    t_dataset_object,
)
from sqlalchemy.orm.exc import NoResultFound


session = get_session(echo=False)

object_dataset = Objects(
    id="55c5b69c-test-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
object_ = Objects(
    id="20720c2e-83fb-4454-bef1-1ce6a97fa748",
    id_type="package",
    id_file=1094489,
    id_internal=None,
)


# ingest = Ingest(session=session)
unit = Units(
    label="test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/test",
)

aspect = Aspects(
    label="diameter-test",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test",
)

desc_inst = DescriptorsInst(
    label="nerve-volume-test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    description=None,
)

desc_quant = DescriptorsQuant(
    shape="scalar",
    label="nerve-volume-test3",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
)

print(list(desc_quant.__mapper__.relationships)[0].direction)
getattr(desc_quant, "aspects", [])

# ingest.insert_objects(
#     objects=[
#         Objects(id="55c5b69c-a5b8-4881-a105-e4048af26fa5", id_type="dataset"),
#     ]
# )
import pandas as pd

df = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
df = df.T.reset_index(drop=True)
df["id_sub"] = "sub-" + df.sub_sam
df.index = df.index + 1
df.head()

RelationshipDirection.MANYTOONE


,NFasc,dFasc_um,dNerve_um,laterality,level,sex,sub_sam,thkperi_um,id_sub
1,15,"[437.21394287682614, 258.82574913412543, 307.9...",2546.924091677267,L,C,M,C50-2,"[24.794722747116708, 27.92527295706188, 17.016...",sub-C50-2
2,5,"[120.12238286446609, 243.86502700380925, 322.2...",2019.0831006899302,L,C,M,C53-2,"[23.91547828345668, 15.931285254737446, 29.499...",sub-C53-2
3,8,"[365.4037755654016, 284.53752037921583, 53.536...",1615.1166896226846,L,C,M,C54-3,"[19.700748058232335, 28.26263525939993, 9.6459...",sub-C54-3
4,1,657.1827113056797,1025.5346591258096,L,C,M,C55-3,"[26.020421901161313, 46.18048430372056, 13.711...",sub-C55-3
5,7,"[455.24233996428916, 509.9466847724346, 287.73...",1838.3739820401397,L,C,M,C57-3,6.81653463390343,sub-C57-3


In [11]:
def object_as_dict(obj):
    return {
        c.key: getattr(obj, c.key)
        for c in inspect(obj).mapper.column_attrs
        if getattr(obj, c.key)
    }


def get_or_create(session, obj, back_populate=None):
    model = obj.__class__
    data = object_as_dict(obj)
    instance = session.query(model).filter_by(**data).one_or_none()
    if instance:
        return instance
    else:
        params = {
            k: v for k, v in data.items() if not isinstance(v, ClauseElement)
        }
        instance = model(**params)
        try:
            session.add(instance)
            session.commit()
            if back_populate:
                for attr, value in back_populate.items():
                    setattr(instance, attr, value)
                session.commit()
        except Exception:
            session.rollback()
            instance = session.query(model).filter_by(**data).one()
            return instance
        else:
            return instance


def get_constraint_columns(model):
    """
    Returns the columns of the unique constraints for a SQLAlchemy model.

    Args:
        model: The SQLAlchemy model.

    Returns:
        A list of lists of column objects.
    """

    constraints = [
        constraint
        for constraint in model.__mapper__.tables[0].constraints
        if isinstance(constraint, UniqueConstraint)
    ]

    return [
        [c.name for c in list(constraint.columns)] for constraint in constraints
    ]


def query_by_constraints(db, obj):
    # print("made it")
    filter_criteria = object_as_dict(obj)

    existing_obj = None
    for cols in get_constraint_columns(obj):

        # print(type(obj), "cols", cols)
        # print("cols", filter_criteria)
        if set(cols) != (set(cols) & set(filter_criteria.keys())):
            continue

        filter_criteria = {
            key: value for key, value in filter_criteria.items() if key in cols
        }

        # print("filter using", filter_criteria)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        if existing_obj:
            # print("found!!")
            return existing_obj
        # print("not found")


def back_populate_tables(db: Session, obj) -> object:
    """
    Recursively back-populates objects in the database,
    retrieving existing objects instead of failing on duplicates.
    Iterates through all relationships of the object for both
    initial population and recursive calls.
    Includes a complete rollback if any commit fails.
    Filters by all primary key columns of the object.
    Handles cases where the child object is not a valid ORM instance.

    Args:
        db: The database session.
        obj: The ORM model object.

    Returns:
        The ORM model object, with its related objects back-populated.
    """

    mapper = obj.__mapper__  # Get the mapper for the object
    # print(obj)
    try:
        # Iterate through all relationships of the object
        for relationship_prop in mapper.relationships:
            if (
                relationship_prop.direction.name == "MANYTOONE"
                or relationship_prop.direction.name == "MANYTOMANY"
            ):
                parent = getattr(obj, relationship_prop.key)
                if parent:
                    filter_criteria = object_as_dict(parent)
                    existing_obj = (
                        db.query(type(parent))
                        .filter_by(**filter_criteria)
                        .first()
                    )
                    if existing_obj:
                        for key, value in object_as_dict(existing_obj).items():
                            if hasattr(parent, key):
                                setattr(parent, key, value)
                    # print("populated", object_as_dict(parent))
                    setattr(obj, relationship_prop.key, parent)
                    # db.add(parent)
                    # db.commit()
                    back_populate_tables(db, parent)

        # print("object", type(obj))
        filter_criteria = object_as_dict(obj)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        print("filter criteria", filter_criteria)

        if not existing_obj:
            existing_obj = query_by_constraints(db, obj)

        # print("filter using", filter_criteria)
        # existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        # print("is existing??", existing_obj)

        if not existing_obj:
            db.merge(obj)
            # db.add(obj)
            # db.flush()
            db.commit()
            # db.refresh(obj)
            # print("after commit", object_as_dict(obj))
        else:
            db.merge(obj)
            db.commit()
            for key, value in object_as_dict(existing_obj).items():
                if hasattr(obj, key):
                    setattr(obj, key, value)
            # print("after update", object_as_dict(existing_obj))

    except IntegrityError as e:
        db.rollback()  # Rollback the transaction in case of an error
        # print(f"Error during commit: {e}")
        raise  # Re-raise the exception to indicate failure

    return obj  # Return the object with its related objects back-populated

In [4]:
# def get_constraint_attribute_names(model_class):
#     """
#     Retrieves all attribute names involved in UniqueConstraints
#     and primary key constraints in a SQLAlchemy declarative base model.

#     Args:
#         model_class: The SQLAlchemy model class.

#     Returns:
#         A list of attribute names.
#     """

#     mapper = model_class.__mapper__
#     constraints = mapper.primary_key + [
#         c for c in mapper.constraints if isinstance(c, UniqueConstraint)
#     ]

#     attribute_names = []
#     for constraint in constraints:
#         for column in constraint.columns:
#             attribute_names.append(column.name)

#     return attribute_names


# def query_by_constraints(db, obj):
#     filter_criteria = object_as_dict(obj)

#     existing_obj = None
#     for cols in get_constraint_columns(obj):

#         if set(cols) != (set(cols) & set(filter_criteria.keys())):
#             continue

#         filter_criteria = {
#             key: value for key, value in filter_criteria.items() if key in cols
#         }

#         existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
#         if existing_obj:
#             return existing_obj


query_by_constraints(session, desc_quant)

made it
<class 'quantdb.ingest_tabular.models.DescriptorsQuant'> cols ['label']
cols {'shape': 'scalar', 'label': 'nerve-volume-test3', 'aggregation_type': 'instance'}
filter using {'label': 'nerve-volume-test3'}
found!!


In [5]:
# Params
object = Objects(
    id="N:dataset:99a9a99a-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

# back_populate_tables(session, object)

In [6]:
# Domain
unit = Units(
    label="test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/test",
)

aspect = Aspects(
    label="diameter-test-backpop4",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test-backpop4",
)
aspect = Aspects(
    label="count",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/count",
)

desc_inst = DescriptorsInst(
    label="nerve-volume-test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    description=None,
)

desc_quant = DescriptorsQuant(
    shape="scalar",
    label="nerve-volume-test8",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)

back_populate_tables(session, desc_inst)
back_populate_tables(session, desc_quant)
# getattr(desc_quant, "aspects")
# list(desc_quant.aspects.__mapper__.relationships)
# back_populate_tables(session, desc_quant)
# print("here", aspect.id)
# print("here", desc_quant.aspects)

object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter criteria {'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
is existing?? <quantdb.ingest_tabular.models.DescriptorsInst object at 0x177723e30>
after update {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
object <class 'quantdb.ingest_tabular.models.Aspects'>
filter criteria {'id': 34, 'label': 'count', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/count'}
is existing?? <quantdb.ingest_tabular.models.Aspects object at 0x177786120>
after update {'id': 34, 'label': 'count', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/count'}
object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter criteria {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
is exist

In [7]:
# Objects

address = Addresses(
    addr_type="constant",
    value_type="single",
)

obj_desc_inst = ObjDescInst(
    objects=object,
    descriptors_inst=desc_inst,
    addresses_field=address,
)

obj_desc_quant = ObjDescQuant(
    objects=object,
    descriptors_quant=desc_quant,
    addresses_field=address,
)

back_populate_tables(session, obj_desc_inst)
# back_populate_tables(session, obj_desc_quant)

object <class 'quantdb.ingest_tabular.models.Addresses'>
filter criteria {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
is existing?? <quantdb.ingest_tabular.models.Addresses object at 0x1777a44d0>
after update {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter criteria {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
is existing?? <quantdb.ingest_tabular.models.DescriptorsInst object at 0x1777a66c0>
after update {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
object <class 'quantdb.ingest_tabular.models.Objects'>
filter criteria {'id': UUID('99a9a99a-a5b8-4881-a105-e4048af26fa5'), 'id_type': 'dataset'}
is existing?? <Objects(id=99a9a99a-a

In [8]:
# Value
values_inst = ValuesInst(
    type="sample",
    id_sub="sub-test",
    id_formal="sam-l-seg-c7-A-level-test",
    local_identifier=None,
    id_sam="sam-l-seg-c7-A-level-test",
    # descriptors_inst=desc_inst,
    # dataset=object.id,
    objects=object,
)
values_quant = ValuesQuant(
    value=0.5,
    objects=object,
    orig_value=None,
    orig_units=None,
    descriptors_inst=desc_inst,
    descriptors_quant=desc_quant,
    values_inst=values_inst,
    value_blob={"key1": "value1"},
)

with session.no_autoflush:
    back_populate_tables(session, values_quant)

object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter criteria {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
is existing?? <quantdb.ingest_tabular.models.DescriptorsInst object at 0x1777bc6b0>
after update {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
object <class 'quantdb.ingest_tabular.models.Aspects'>
filter criteria {'id': 34, 'label': 'count', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/count'}
is existing?? <quantdb.ingest_tabular.models.Aspects object at 0x1777be480>
after update {'id': 34, 'label': 'count', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/count'}
object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter criteria {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'

In [3]:
# ingest.insert_objects(
#     objects=[
#         Objects(id="55c5b69c-a5b8-4881-a105-e4048af26fa5", id_type="dataset"),
#     ]
# )
import pandas as pd

df = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
df = df.T.reset_index(drop=True)
df["id_sub"] = "sub-" + df.sub_sam
df.index = df.index + 1
df.head()

,NFasc,dFasc_um,dNerve_um,laterality,level,sex,sub_sam,thkperi_um,id_sub
1,15,"[437.21394287682614, 258.82574913412543, 307.9...",2546.924091677267,L,C,M,C50-2,"[24.794722747116708, 27.92527295706188, 17.016...",sub-C50-2
2,5,"[120.12238286446609, 243.86502700380925, 322.2...",2019.0831006899302,L,C,M,C53-2,"[23.91547828345668, 15.931285254737446, 29.499...",sub-C53-2
3,8,"[365.4037755654016, 284.53752037921583, 53.536...",1615.1166896226846,L,C,M,C54-3,"[19.700748058232335, 28.26263525939993, 9.6459...",sub-C54-3
4,1,657.1827113056797,1025.5346591258096,L,C,M,C55-3,"[26.020421901161313, 46.18048430372056, 13.711...",sub-C55-3
5,7,"[455.24233996428916, 509.9466847724346, 287.73...",1838.3739820401397,L,C,M,C57-3,6.81653463390343,sub-C57-3


In [11]:
# number of fascicles

# Dataset
object_dataset = Objects(
    id="55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
object_package = Objects(
    id="20720c2e-83fb-4454-bef1-1ce6a97fa748",
    id_type="package",
    id_file=1094489,
    id_internal=None,
    objects_=object_dataset,
)

# Domain
unit = Units(
    label="unitless",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/micrometer",
)

aspect = Aspects(
    label="count",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/count",
)
# focus of measurement
desc_inst = DescriptorsInst(
    label="nerve-cross-section",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/cross-section",
    # description=None,
)

# how to calculate unit
desc_quant = DescriptorsQuant(
    shape="scalar",
    label="count of fascicles",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)

# back_populate_tables(session, desc_inst)
# back_populate_tables(session, desc_quant)

# Objects
address = Addresses(
    addr_type="constant",
    value_type="single",
)
obj_desc_inst = ObjDescInst(
    objects=object_package,
    descriptors_inst=desc_inst,
    addresses_field=address,
)
obj_desc_quant = ObjDescQuant(
    objects=object_package,
    descriptors_quant=desc_quant,
    addresses_field=address,
)

# values type
controlled_term = ControlledTerms(
    label="controlled thing",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
)

# back_populate_tables(session, obj_desc_inst)
# back_populate_tables(session, obj_desc_quant)

for i, row in df.iterrows():
    id_sub = "sub-" + row.sub_sam[1:].split("-")[0]
    id_sam = "sam-" + row.sub_sam[1:]
    id_formal = id_sam
    # try:
    #     # Check if the entry already exists
    #     existing_values_inst = (
    #         session.query(ValuesInst)
    #         .filter_by(
    #             id_sub=id_sub,
    #             id_formal=id_formal,
    #             id_sam=id_sam,
    #             objects=object_package,
    #         )
    #         .one()
    #     )
    #     values_inst = existing_values_inst
    # except NoResultFound:
    #     # If not found, create a new entry
    #     values_inst = ValuesInst(
    #         dataset=object_dataset,
    #         desc_inst=desc_inst,
    #         type="sample",
    #         id_sub=id_sub,
    #         id_sam=id_sam,
    #         id_formal=id_formal,
    #         local_identifier=id_sam,
    #         objects=object_package,
    #     )
    #     session.add(values_inst)
    #     session.commit()

    values_quant = ValuesQuant(
        value=int(row["NFasc"]),
        objects=object_package,
        orig_value=None,
        orig_units=None,
        descriptors_inst=desc_inst,
        descriptors_quant=desc_quant,
        values_inst=values_inst,
        value_blob=row["NFasc"],
    )

    back_populate_tables(session, values_quant)
# # values to iterate through
# values_inst = ValuesInst(
#     type="sample",
#     id_sub="sub-",
#     id_formal="",
#     local_identifier=None,
#     id_sam="",
#     # descriptors_inst=desc_inst,
#     # dataset=object.id,
#     objects=object,
# )
# values_quant = ValuesQuant(
#     value=0.5,
#     objects=object,
#     orig_value=None,
#     orig_units=None,
#     descriptors_inst=desc_inst,
#     descriptors_quant=desc_quant,
#     values_inst=values_inst,
#     value_blob={"key1": "value1"},
# )

# back_populate_tables(session, values_quant)

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'DescriptorsInst'
[SQL: INSERT INTO values_inst (type, id_sub, desc_inst, dataset, id_formal, local_identifier, id_sam) VALUES (%(type)s, %(id_sub)s, %(desc_inst)s, %(dataset)s::UUID, %(id_formal)s, %(local_identifier)s, %(id_sam)s) RETURNING values_inst.id]
[parameters: {'type': 'sample', 'id_sub': 'sub-50', 'desc_inst': <quantdb.ingest_tabular.models.DescriptorsInst object at 0x17b87d0a0>, 'dataset': '20720c2e-83fb-4454-bef1-1ce6a97fa748', 'id_formal': 'sam-50-2', 'local_identifier': 'sam-50-2', 'id_sam': 'sam-50-2'}]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [5]:
object = Objects(
    id="N:dataset:55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
back_populate_tables(session, object)

filter criteria {'id': '55c5b69c-a5b8-4881-a105-e4048af26fa5', 'id_type': 'dataset'}


<Objects(id=55c5b69c-a5b8-4881-a105-e4048af26fa5, id_type=dataset)>

In [6]:
# dfasc
# session = get_session(echo=False)
# Dataset
object = Objects(
    id="N:dataset:55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
back_populate_tables(session, object)
# Domain
unit = Units(
    label="um",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/micrometer",
)
aspect = Aspects(
    label="diameter",
    description=None,
    # iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/count",
)
# focus of measurement
desc_inst = DescriptorsInst(
    label="fascicle-cross-section",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    # description=None,
)
# back_populate_tables(session, desc_inst)
# how to calculate unit
desc_quant = DescriptorsQuant(
    shape="scalar",
    label="fascicle cross section diameter um",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)

# Objects
address = Addresses(
    id=2,
    addr_type="constant",
    value_type="single",
)
obj_desc_inst = ObjDescInst(
    objects=object,
    descriptors_inst=desc_inst,
    addr_field=25,
    addresses_field=address,
)
obj_desc_quant = ObjDescQuant(
    objects=object,
    addr_field=6,
    descriptors_quant=desc_quant,
    addresses_field=address,
)
# try:
#     session.add(obj_desc_quant)
#     session.commit()
# except:
#     pass
back_populate_tables(session, obj_desc_inst)
back_populate_tables(session, obj_desc_quant)

# Ensure obj_desc_inst is added to the session and committed
# session.add(obj_desc_inst)
# session.commit()

for i, row in df.iterrows():
    id_sub = "sub-" + row.sub_sam[1:].split("-")[0]
    id_sam = "sam-" + id_sub + "_sam-" + row.sub_sam[1:].split("-")[-1]
    focus_values = ast.literal_eval(row["dFasc_um"])
    if not isinstance(focus_values, list):
        focus_values = [focus_values]
    print(i)
    for j, value in enumerate(focus_values):

        id_formal = "fasc-" + id_sam + f"-{j}"
        values_inst = ValuesInst(
            type="below",
            id_sub=row.id_sub,
            id_sam=id_sam,
            id_formal=id_formal,
            local_identifier=None,
            objects=object,
            desc_inst=desc_inst.id,  # Use the primary key or appropriate attribute
        )
        back_populate_tables(session, values_inst)
        try:
            # Check if the entry already exists
            existing_values_inst = (
                session.query(ValuesInst)
                .filter_by(
                    id_sub=row.id_sub,
                    id_formal=id_formal,
                    id_sam=id_sam,
                    objects=object,
                )
                .one()
            )
            values_inst = existing_values_inst
        except NoResultFound:
            # If not found, create a new entry
            values_inst = ValuesInst(
                type="below",
                id_sub=row.id_sub,
                id_sam=id_sam,
                id_formal=id_formal,
                local_identifier=None,
                objects=object,
                desc_inst=desc_inst,
            )
            session.add(values_inst)
            session.commit()
            # back_populate_tables(session, values_inst)

        # print("values!", value)
        if not isinstance(value, float):
            continue
        values_quant = ValuesQuant(
            value=value,
            objects=object,
            orig_value=None,
            orig_units=None,
            desc_inst=25,
            descriptors_inst=desc_inst,
            descriptors_quant=desc_quant,
            values_inst=values_inst,
            value_blob=str(value),
            # obj_desc_inst=obj_desc_inst,
            # obj_desc_inst=obj_desc_quant,
        )

        # back_populate_tables(session, values_quant)

filter criteria {'id': '55c5b69c-a5b8-4881-a105-e4048af26fa5', 'id_type': 'dataset'}
filter criteria {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
filter criteria {'id': 25, 'label': 'fascicle-cross-section', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/fascicle/cross-section'}
filter criteria {'id': UUID('55c5b69c-a5b8-4881-a105-e4048af26fa5'), 'id_type': 'dataset'}
filter criteria {'object': UUID('55c5b69c-a5b8-4881-a105-e4048af26fa5'), 'desc_inst': 25, 'addr_field': 25}
filter criteria {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
filter criteria {'id': 5, 'label': 'diameter', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter'}
filter criteria {'id': 25, 'label': 'fascicle-cross-section', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/fascicle/cross-section'}
filter criteria {'id': 6, 'shape': '

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "values_inst_dataset_id_formal_key"
DETAIL:  Key (dataset, id_formal)=(55c5b69c-a5b8-4881-a105-e4048af26fa5, fasc-sam-sub-50_sam-2-0) already exists.

[SQL: INSERT INTO values_inst (type, id_sub, desc_inst, dataset, id_formal, local_identifier, id_sam) VALUES (%(type)s, %(id_sub)s, %(desc_inst)s, %(dataset)s::UUID, %(id_formal)s, %(local_identifier)s, %(id_sam)s) RETURNING values_inst.id]
[parameters: {'type': 'below', 'id_sub': 'sub-C50-2', 'desc_inst': 25, 'dataset': UUID('55c5b69c-a5b8-4881-a105-e4048af26fa5'), 'id_formal': 'fasc-sam-sub-50_sam-2-0', 'local_identifier': None, 'id_sam': 'sam-sub-50_sam-2'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

# dream ingest

In [4]:
# ingest.insert_descriptors_quant(
#     descriptor_quants=[
#         DescriptorQuant(
#             label="fascicle cross section diameter um min",
#             domain=26,  # "fiber-cross-section" # TODO: label to ID
#             aspect=5,  # "diameter",  # TODO: label to ID
#             unit=4,  #  pixel",  # TODO: label to ID
#             aggregation_type="min",
#         ),
#     ]
# )

# # dream ingest
# csv = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
# ingest.insert_descriptors_quant(**csv.to_dict(orient="records"))